<a href="https://colab.research.google.com/github/Pooja-Shanbhag/Predicting-Iris-Flower-Species/blob/master/Iris%20Species.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#imports
from torch import Tensor
from torch.nn import Module
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torch.optim import SGD
from numpy import vstack
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from pandas import read_csv

In [15]:
class CSVDataset(Dataset):
    def __init__(self,path):
        dataframe = read_csv(path,header = None)
        self.X = dataframe.values[:,:-1]
        self.X = self.X.astype('float32')
        self.y = dataframe.values[:,-1]
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('long')
        self.y = self.y.reshape(len(self.y),1)
        
    def __getitem__(self,idx):
        return [self.X[idx],self.y[idx]]
    
    def __len__(self):
        return len(self.X)
    
    def split_data(self, n_train = 0.67):
        train_len = round(len(self.X)*n_train)
        test_len = len(self.X) - train_len
        return random_split(self, [train_len,test_len])

In [3]:
class MLP(Module):
    def __init__(self,inputs):
        super(MLP,self).__init__()
        self.hidden1 = Linear(inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity = 'relu')
        self.act1 = ReLU()
        
        self.hidden2 = Linear(10,8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity = 'relu')
        self.act2 = ReLU()
        
        self.hidden3 = Linear(8,3)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Softmax(dim=1)
        
    def forward(self,X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        return X

In [4]:
def prepare_data(path):
    dataset = CSVDataset(path)
    train_data, test_data = dataset.split_data()
    train_dl = DataLoader(train_data, batch_size = 32, shuffle = True)
    test_dl = DataLoader(test_data, batch_size = 1024, shuffle = False)
    return train_dl, test_dl

In [18]:
def train_model(model,train_dl):
    criterion = CrossEntropyLoss()
    optim = SGD(model.parameters(), lr = 0.01, momentum = 0.9)
    for epoch in range(600):
        for i,(x,y) in enumerate(train_dl):
            optim.zero_grad()
            yhat = model(x)
            loss = criterion(yhat,y)
            loss.backward()
            optim.step()


In [10]:
def evaluate_model(model,test_dl):
    pred, actuals = list(), list()
    for i,(x,y) in enumerate(test_dl):
        yhat = model(x)
        
        yhat = yhat.detach().numpy()
        yhat = argmax(yhat,axis = 1)
        yhat = yhat.reshape(len(yhat),1)
        
        actual = y.numpy()
        actual = actual.reshape(len(actual),1)
        
        pred.append(yhat)
        actuals.append(y)
    pred, actuals = vstack(pred), vstack(actuals)
    acc = accuracy_score(actuals,pred)
    return acc
        
            

In [11]:
def predict_model(row,model):
    row = Tensor([row])
    yhat = model(row)
    yhat = yhat.detach().numpy()
    return yhat    

In [20]:
path = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
train_dl, test_dl = prepare_data(path)
model = MLP(4)
train_model(model,train_dl)
acc = evaluate_model(model,test_dl)
print("accuracy ",acc)

ValueError: Target and input must have the same number of elements. target nelement (32) != input nelement (96)

In [2]:
# pytorch mlp for multiclass classification
from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 3)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Softmax(dim=1)

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # output layer
        X = self.hidden3(X)
        X = self.act3(X)
        return X

# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(500):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # convert to class labels
        yhat = argmax(yhat, axis=1)
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

# prepare the data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(4)
# train the model
train_model(train_dl, model)
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)
# make a single prediction
row = [5.1,3.5,1.4,0.2]
yhat = predict(row, model)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

100 50


RuntimeError: Expected object of scalar type Long but got scalar type Int for argument #2 'target' in call to _thnn_nll_loss_forward